In [5]:
import seekpath
import numpy as np

# Define the `kVector` class

In [27]:
class kVector:
    """For k-vector search, given the input structure, the nucleus and satellite
    diffraction peaks.

    :param bravfSym: Bravais lattice symbol
    :param cell: a `3x3` list of floats (cell[0] is the first lattice vector, …)
                     The cell vectors should be corresponding to the primitive
                     following the ITA convention.
    :param positions: a `Nx3` list of floats with the atomic coordinates in
                      fractional coordinates (i.e., w.r.t. the cell vectors)
    :param numbers: a length-`N` list with integers identifying uniquely the
                    atoms in the cell
    :param nucPeaks: a `mx4` list for holding the nucleus diffraction peaks
                     (nucPeaks[0] has 4 entries, giving the hkl indeces
                     corresponding to the conventional unit cell setting (ITA),
                     together with the d-spacing)
    :param superPeaks: a length-`n` list for the satellite peak positions in
                       `d`.
    :param threshold: specify the delta_d/d threshold. When the distance
                      between two peaks is smaller than such a threshold, they
                      would then be considered as identical. When searching
                      for the single k vector, if the maximum value among those
                      distances between the nominal and observed positions of
                      the satellite peaks is smaller than the threshold, the
                      corresponding k vector will be returned directly as the
                      optimal solution.
    :param option: (optional) control the scope for k vector search in the
                   Brillouin zone, `0` for high symmetry points only, `1` for
                   high symmetry and edges, `2` for the whole Brillouin zone.
    """
    transMatrix = {
        "cP": np.array(
            [
                [1, 0, 0],
                [0, 1, 0],
                [0, 0, 1]
            ]
        ),
        "tP": np.array(
            [
                [1, 0, 0],
                [0, 1, 0],
                [0, 0, 1]
            ]
        ),
        "hP": np.array(
            [
                [1, 0, 0],
                [0, 1, 0],
                [0, 0, 1]
            ]
        ),
        "oP": np.array(
            [
                [1, 0, 0],
                [0, 1, 0],
                [0, 0, 1]
            ]
        ),
        "mP": np.array(
            [
                [1, 0, 0],
                [0, 1, 0],
                [0, 0, 1]
            ]
        ),
        "cF": 1/2 * np.array(
            [
                [0, 1, 1],
                [1, 0, 1],
                [1, 1, 0]
            ]
        ),
        "oF": 1/2 * np.array(
            [
                [0, 1, 1],
                [1, 0, 1],
                [1, 1, 0]
            ]
        ),
        "cI": 1/2 * np.array(
            [
                [-1, 1, 1],
                [1, -1, 1],
                [1, 1, -1]
            ]
        ),
        "tI": 1/2 * np.array(
            [
                [-1, 1, 1],
                [1, -1, 1],
                [1, 1, -1]
            ]
        ),
        "oI": 1/2 * np.array(
            [
                [-1, 1, 1],
                [1, -1, 1],
                [1, 1, -1]
            ]
        ),
        "hR": 1/3 * np.array(
            [
                [2, -1, -1],
                [1, 1, -2],
                [1, 1, 1]
            ]
        ),
        "oC": 1/2 * np.array(
            [
                [1, 1, 0],
                [-1, 1, 0],
                [0, 0, 2]
            ]
        ),
        "oA": 1/2 * np.array(
            [
                [0, 0, 2],
                [1, 1, 0],
                [-1, 1, 0]
            ]
        ),
        "mC": 1/2 * np.array(
            [
                [1, -1, 0],
                [1, -1, 1],
                [1, 1, -1]
            ]
        )
    }

    
    def __init__(
            self,
            bravfSym: str,
            cell: list,
            positions: list,
            numbers: list,
            nucPeaks: list,
            superPeaks: list,
            threshold: float,
            option: int = 0
        ):
        self.bravfSym = bravfSym
        self.cell = cell
        self.positions = positions
        self.numbers = numbers
        self.nucPeaks = nucPeaks
        self.superPeaks = superPeaks
        self.threshold = threshold
        self.option = option
        

    def kpathFinder(self) -> dict:
        """Provided the structure inputs, the routine will be collecting the
        inputs into a structure tuple which will be fed into the `get_path`
        routine in the `seekpath` module. The special k points and the k-path
        will be returned.
        
        :return: the dictionary containing the k-points, k-path and the
                 reciprocal space primitive lattice vectors. 
        """
        structure = (self.cell, self.positions, self.numbers)
        k_info_tmp = seekpath.get_path(structure, True)
        k_info = {
            "point_coords": k_info_tmp["point_coords"],
            "path": k_info_tmp["path"],
            "reciprocal_primitive_lattice": k_info_tmp[
                "reciprocal_primitive_lattice"
            ]
        }
    
        return k_info


    def hklConvToPrim(self, hkl: list) -> list:
        """Convert the hkl indeces in the conventional cell setting to the
        primitive cell setting.

        :param hkl: input hkl indeces in the conventional cell setting
        :return: a list containing the hkl indeces in the primitive cell
                 setting.
        """
        prim_hkl = np.matmul(
            np.array([hkl]),
            kVector.transMatrix[self.bravfSym]
        )

        return list(prim_hkl)


    def pointOnVector(s_point: list, e_point: list, distance: float) -> list:
        """Grab the coordinate of a point on a vector specified by the starting
        and ending points. The distance from the point on the vector to the
        starting point should be given as the parameter.

        :param s_point: a list for the coordinate of the starting point
        :param e_point: a list for the coordinate of the ending point
        :param distance: the distance away from the starting point
        :return: the coordinate of the point on the vector
        """
        vec_length = np.sqrt(sum((x - y)**2 for x, y in zip(s_point, e_point)))

        xp = s_point[0] + distance / vec_length * (e_point[0] - s_point[0])
        yp = s_point[1] + distance / vec_length * (e_point[1] - s_point[1])
        zp = s_point[2] + distance / vec_length * (e_point[2] - s_point[2])

        return [xp, yp, zp]


    def insIntoSortedList(lst: list, new_val: float) -> tuple:
        """Insert a new entry into the ascendingly sorted list.
        
        :param lst: an ascendingly sorted list
        :param new_val: the new entry to be inserted into the sorted list
        :return: tuple containing the new sorted list and the index of the new
                entry in the new sorted list.
        """
        ind = 0
        for i in range(len(lst)):
            if lst[i] < new_val:
                ind = i + 1
            else:
                break
        lst.insert(ind, new_val)

        return (lst, ind)


    def updateCandidateList(
            self,
            kpoint: list,
            k_opt_list: list,
            k_opt_dist:list
        ) -> tuple:
        """For a given k point, we want to first cycle through all the
        satelliete peaks. For each satellite peak, we want to find out which
        nucleus peak that the satellite peak should be attached to, meaning the
        nucleus peak for which the hkl plus the trial k vector would yield the
        minimum distance from the observed peak position of the satellite peak
        under consideration. The maximum value among the minimum distances for
        all the satellite peaks will be taken as the indicator for whether the
        given k point is a good candidate. The new `indicator distance` will be
        inserted into the indicator distance list of those top candidates and
        the given k point will be inserted into the corresponding location in
        the top candidates list of the k vectors. If the `indicator distance`
        happens to be smaller than the uncertainty of peak positions (which is
        determined by the instrument resolution), only the top candidate will be
        returned.

        :param kpoint: the trial k vector
        :param k_opt_list: list of top candidates of k vectors
        :param k_opt_dist: list of the `indicator distances` of those top
                           candidates of k vectors
        :return: tuple of the updated list of top candidates of k vectors and
                 the updated list of the `indicator distances`
        """
        rep_prim_latt = self.kpathFinder()["reciprocal_primitive_lattice"]

        speaks_closest_dist = list()
        for sp in self.superPeaks:
            dist_min = np.Inf
            for nucp in self.nucPeaks:
                hkl_conv = nucp[:3]
                hkl_prim = self.hklConvToPrim(hkl_conv)
                sp_norm = np.array(
                    [
                        [
                            hkl_prim[0] + kpoint[0],
                            hkl_prim[1] + kpoint[1],
                            hkl_prim[2] + kpoint[2]
                        ]
                    ]
                )
                sp_norm_cart = np.matmul(
                    sp_norm,
                    rep_prim_latt
                )
                sp_norm_d = 2. * np.pi / np.linalg.norm(sp_norm_cart)
                sp_rel_diff = abs(sp_norm_d - sp) / sp
                if sp_rel_diff < dist_min:
                    dist_min = sp_rel_diff
            speaks_closest_dist.append(dist_min)

        if max(speaks_closest_dist) <= self.threshold:
            k_opt_list = [kpoint]
            k_opt_dist = [max(speaks_closest_dist)]
            return (k_opt_list, k_opt_dist)
        else:
            k_opt_dist, ind = self.insIntoSortedList(
                k_opt_dist,
                max(speaks_closest_dist)
            )
            k_opt_list.insert(ind, kpoint)

            return (k_opt_list[:10], k_opt_dist[:10])


    def kOptFinder(self) -> list:
        """This is the kernel of the class, defining the method for searching
        over the Brillouin zone for optimal k vector that best explains the
        observed satellite peaks, given the already refined nucleus structure.

        :return: the list of top candidates of the optimal k vector. If the top
                 one is below the threshold, i.e., the distance between the
                 nominal and observed positions of those satellite peaks smaller
                 than the uncertainty of peak positions (which is determined by
                 the instrument resolution), only one candiate will be returned.
                 Otherwise, top 10 candidates will be returned. 
        """
        # First, we search over the high symmetry points
        hs_points = self.kpathFinder()["point_coords"]
        
        k_opt_list = list()
        k_opt_dist = list()

        if len(self.nucPeaks) < np.floor(len(self.superPeaks) / 2):
            err_msg = "The number of nucleus peaks is not enough for "
            err_msg += "k vector determination."
            raise ValueError("err_msg")
        else:
            if len(self.nucPeaks) >= len(self.superPeaks):
                for _, kpoint in hs_points.items():
                    k_opt_tmp = self.updateCandidateList(
                        kpoint,
                        k_opt_list,
                        k_opt_dist
                    )
                    k_opt_list = k_opt_tmp[0]
                    k_opt_dist = k_opt_tmp[1]

                    if len(k_opt_list) == 1:
                        return k_opt_list

                if self.option == 0:
                    return k_opt_list
            else:
                if self.option == 0:
                    err_msg = "The number of nucleus peaks is less than that "
                    err_msg += "of the satellite peaks, and thus the search "
                    err_msg += "should go beyond the high symmetry point. \n"
                    err_msg += "Please select to search either along k paths "
                    err_msg += "or over the whole 1st Brillouin zone."
                    raise ValueError(err_msg)
                else:
                    warn_msg = "The number of nucleus peaks is less than that "
                    warn_msg += "of the satellite peaks, thus skipping the "
                    warn_msg += "search over the high symmetry points."
                    print(f"[Warning] {warn_msg}")

# Testing

In [8]:
cell = [
    [2.0626889900, 2.6083333300, -2.0959087400],
    [1.9072983400, 2.9395873000, 1.8656656300],
    [6.5783827900, -3.2025691800, 1.4025826200]
]
positions = [
    [0.2293290750, 0.9643429050, 0.7541443700],
    [0.7706709250, 0.0356570950, 0.2458556300],
    [0.1294345050, 0.4304467200, 0.0623625850],
    [0.8705654950, 0.5695532800, 0.9376374150],
    [0.4456382750, 0.6359233100, 0.4306348850],
    [0.5543617250, 0.3640766900, 0.5693651150]
]
numbers = [i for i in range(len(positions))]

nucPeaks = [
    [0, 1, 1, 2.2]
]
superPeaks = [
    2.25
]

In [55]:
k_search = kVector(
    "cP",
    cell,
    positions,
    numbers,
    nucPeaks,
    superPeaks
)

In [57]:
k_path = k_search.kpathFinder()

In [58]:
k_path

{'point_coords': {'GAMMA': [0.0, 0.0, 0.0],
  'Z': [0.0, 0.0, 0.5],
  'Y': [0.0, 0.5, 0.0],
  'Y_2': [0.0, -0.5, 0.0],
  'X': [0.5, 0.0, 0.0],
  'V_2': [0.5, -0.5, 0.0],
  'U_2': [-0.5, 0.0, 0.5],
  'T_2': [0.0, -0.5, 0.5],
  'R_2': [-0.5, -0.5, 0.5]},
 'path': [('GAMMA', 'X'),
  ('Y', 'GAMMA'),
  ('GAMMA', 'Z'),
  ('R_2', 'GAMMA'),
  ('GAMMA', 'T_2'),
  ('U_2', 'GAMMA'),
  ('GAMMA', 'V_2')],
 'reciprocal_primitive_lattice': [[-0.0, 0.0, -0.8600062951041388],
  [-1.5984632915952437, 0.9054990485017332, -0.038955684089450014],
  [-0.0, 1.8190364917920923, -0.3327601668007708]]}